# Neural Network Components

Softmax applies the exponential to each element and normalizes to get probabilities.

## Softmax

In [1]:
import torch
import torch.nn as nn
import numpy as np

In [2]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [ ]:
x = np.array([2.0, 1.0, 0.1])
outputs = softmax(x)
print(f'data: {x}')
print(f'softmax: {outputs}')

In [6]:
x = torch.tensor([2.0, 1.0, 0.1])
outputs = torch.softmax(x, dim=0)
print(f'softmax: {outputs}')

softmax: tensor([0.6590, 0.2424, 0.0986])


## Cross-Entropy

Takes class probability prediction and outputs lower loss if they are close to true class labels. Labels must be one-hot encoded.

In [7]:
def cross_entropy(actual, predicted):
    loss = -np.sum(actual * np.log(predicted))
    return loss

In [9]:
# Labels must be one-hot encoded
y_true = np.array([1, 0, 0])
y_good = np.array([0.7, 0.2, 0.1])
y_bad = np.array([0.1, 0.3, 0.6])

In [10]:
loss1 = cross_entropy(y_true, y_good)
loss2 = cross_entropy(y_true, y_bad)
print(f'loss1 entropy: {loss1:.4f}')
print(f'loss2 entropy: {loss2:.4f}')

loss1 entropy: 0.3567
loss2 entropy: 2.3026


**PyTorch**

In [15]:
# Torch ce loss applies log softmax and neg. log likelihood
# Don't use softmax in the last layer
loss = nn.CrossEntropyLoss()

In [16]:
# Y true should use class labels, NOT one-hot encodings
# Y predictions have raw scores (logits), not softmax
y_true = torch.tensor([0])

# Predictions have dimensions n_samples x n_classes (1x3)
y_good = torch.tensor([[2.0, 1.0, 0.1]])
y_bad = torch.tensor([[0.5, 2.0, 0.3]])

In [17]:
loss1 = loss(y_good, y_true)
loss2 = loss(y_bad, y_true)

In [18]:
print(f'loss1 entropy: {loss1:.4f}')
print(f'loss2 entropy: {loss2:.4f}')

loss1 entropy: 0.4170
loss2 entropy: 1.8406


In [19]:
_, pred1 = torch.max(y_good, 1)
_, pred2 = torch.max(y_bad, 1)
print(pred1)
print(pred2)

tensor([0])
tensor([1])


In [20]:
# You can also pass in multiple samples at once to compute loss (batching)
y_true = torch.tensor([2, 0, 1])
y_good = torch.tensor([
    [0.1, 1.0, 2.1],
    [2.0, 1.0, 0.1],
    [0.1, 3.0, 0.1]
])
y_bad = torch.tensor([
    [2.1, 1.0, 0.1],
    [0.1, 1.0, 2.1],
    [0.1, 3.0, 0.1]
])

In [21]:
loss1 = loss(y_good, y_true)
loss2 = loss(y_bad, y_true)

In [22]:
print(f'loss1 entropy: {loss1:.4f}')
print(f'loss2 entropy: {loss2:.4f}')

loss1 entropy: 0.3018
loss2 entropy: 1.6242


In [24]:
_, pred1 = torch.max(y_good, 1)
_, pred2 = torch.max(y_bad, 1)
print(f'true labels: {y_true}')
print(f'pred1 labels: {pred1}')
print(f'pred2 labels: {pred2}')

true labels: tensor([2, 0, 1])
pred1 labels: tensor([2, 0, 1])
pred2 labels: tensor([0, 2, 1])


## Neural Networks

In [31]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.linear2(out) 
        
        # Return with no softmax
        return out

In [32]:
model = NeuralNet(input_size = 28*28, hidden_size = 5, num_classes=3)

# Cross-entropy loss applies the softmax
criterion = nn.CrossEntropyLoss()

In [30]:
# For binary classification, use sigmoid function and BCELoss

In [34]:
class BCNeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(BCNeuralNet, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.linear2(out) 
        y_pred = torch.sigmoid(out)
        return out

In [35]:
model = BCNeuralNet(input_size = 28*28, hidden_size = 5)
criterion = nn.BCELoss()

## Activations

Activation functions apply a non-linear transformation and decide whether a neuron should be activated or not. We need non-linearities in the network, otherwise the entire network would be a linear regression model.

In [36]:
# Step function is a simple threshold
# Sigmoid outputs probability between 0 and 1
# Hyperbolic tangent is scaled sigmoid, shifted to -1 to +1 (hidden)
# ReLU outputs 0 for negative, input as output for >0
# Leaky ReLU, minimize input for negative (vanishing gradient)
# Softmax squashes logits (scores) into probabilities (last layer)

In [38]:
class NeuralNetV1(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(NeuralNetV1, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.linear1 = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.linear2(out)
        out = self.sigmoid(out)
        return out

In [40]:
class NeuralNetV2(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(NeuralNetV2, self).__init__()
        # Or, only define linear layers
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.linear1 = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        # And apply activations from torch afterwards
        out = torch.relu(self.linear1(x))
        out = torch.sigmoid(self.linear2(out))
        return out